In [1]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

from concurrent import futures
import numpy as np
from scipy.stats import gaussian_kde
import pandas_datareader.data as web
import yfinance as yf

data_dir = "./data/stock_data_for_finding_attractive_stocks"
os.makedirs(data_dir, exist_ok=True)

current_day = str(dt.date.today()).replace('-','')
start_time = dt.date(1980, 1, 1)
end_time = dt.date.today()

In [2]:
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
sp500_df = tables[0]
second_table = tables[1]
print(sp500_df.shape)
sp500_df["Symbol"] = sp500_df["Symbol"].map(lambda x: x.replace(".", "-"))  # rename symbol to escape symbol error
sp500_df.to_csv("./data/SP500_%s.csv" % current_day, index=False)
sp500_df = pd.read_csv("./data/SP500_%s.csv" % current_day)
print(sp500_df.shape)
sp500_tickers = list(sp500_df["Symbol"])
sp500_df.head()

(504, 9)
(504, 9)


,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981


In [3]:
def download_stock(stock):
    try:
        stock_df = yf.download(stock, start_time, end_time)
        print(stock_df)
        stock_df['Name'] = stock
        output_name = f"{data_dir}/{stock}.csv"
        stock_df.to_csv(output_name)
    except:
        bad_names.append(stock)
        print('bad: %s' % (stock))

        
""" set the download window """
bad_names =[] #to keep track of failed queries
#set the maximum thread number
max_workers = 100000
#now = dt.datetime.now()
path_failed_queries = f'{data_dir}/failed_queries.txt'

if os.path.exists(path_failed_queries):
    with open(path_failed_queries) as f:
        failed_queries = f.read().split("\n")[:-1]
        sp500_tickers_ = failed_queries

else:
    sp500_tickers_ = sp500_tickers

print("number of stocks to download:", len(sp500_tickers_))
#workers = min(max_workers, len(sp500_tickers_)) #in case a smaller number of stocks than threads was passed in
workers = max_workers

with futures.ThreadPoolExecutor(workers) as executor:
    res = executor.map(download_stock, sp500_tickers_)

""" Save failed queries to a text file to retry """
if len(bad_names) > 0:
    with open(path_failed_queries, 'w') as outfile:
        for name in bad_names:
            outfile.write(name+'\n')

while len(bad_names) > 0:
    with futures.ThreadPoolExecutor(workers) as executor:
        res = executor.map(download_stock, sp500_tickers_)
#finish_time = dt.datetime.now()
#duration = finish_time - now
#minutes, seconds = divmod(duration.seconds, 60)
#print(f'The threaded script took {minutes} minutes and {seconds} seconds to run.')
print(f"{len(bad_names)} stocks failed: ", bad_names)

number of stocks to download: 453
[*********************100%***********************]  1 of 1 completed
bad: EBAY
[*********************100%***********************]  1 of 1 completed


bad: FANG


bad: MDLZ



bad: ISRG

bad: MMM

bad: DIS
[*********************100%***********************]  1 of 1 completed




bad: ZBH




bad: ILMN



bad: K

bad: DOV








bad: MRK




bad: HPQ








bad: ZION



bad: ADI

bad: ES

bad: KR









bad: PSA


bad: LRCX

bad: KEY














bad: APA



bad: RL










bad: EMR

bad: USB








bad: ADP






[*********************100%***********************]  1 of 1 completed

bad: COO

bad: BBWI

bad: DGX

bad: CDNS


bad: UPS

bad: UNH



bad: PPL

bad: BLK



bad: INCY







bad: TTWObad: ALL



bad: EVRGbad: RCLbad: PLD

bad: LYB






bad: MSCIbad: AJGbad: LNTbad: FCXbad: OTIS



bad: XOM

bad: UDR
bad: MAbad: EOGbad: SWKSbad: ABMD

bad: FTV
bad: CTSH




bad: APH
bad: MTBbad: CMSbad: PRUbad: NTAPbad: PFE


bad: NOC

bad: EMN
bad: SWK


In [21]:
historical_stock_data_files = glob.glob(f"C:/Usersfilro/Documents/Stocks/data/SP500_20220430.csv")
highest_day_list = []

for files in historical_stock_data_files:
    price = pd.read_csv(files, index_col="Date", parse_dates=True)
    ticker = os.path.splitext(os.path.basename(files))[0]
    price_close = price[["Close"]]
    highest_day = price_close.idxmax()[0]
    highest_price = price_close.max()[0]
    highest_day_list.append(pd.DataFrame({"highest_day": [highest_day], "ticker": [ticker], "highest_price": highest_price}))
        
    
    
df = pd.concat(highest_day_list).reset_index(drop=True)
print(df.shape)
df.head()

ValueError: No objects to concatenate

In [9]:
# additional info
df["highest_month"] = df["highest_day"].dt.to_period("M")
df = pd.merge(df, sp500_df[["Symbol", "GICS Sector", "GICS Sub-Industry"]], left_on='ticker', right_on='Symbol')

NameError: name 'df' is not defined

In [ ]:
df.sort_values("highest_day",ascending=False).head(20)

In [ ]:
industry_value_counts = df[df["highest_day"] >= "2022-04-01"]["GICS Sub-Industry"].value_counts()
fig, ax = plt.subplots(figsize=(20, 8))
ax.bar(industry_value_counts.index, industry_value_counts.values)
ax.set_xticklabels(industry_value_counts.index, rotation=90)
ax.set_xlabel("industry")
ax.set_ylabel("number of stocks")
plt.show()

In [ ]:
display(industry_value_counts[industry_value_counts.index.str.contains("Utilities")])
display(industry_value_counts[industry_value_counts.index.str.contains("Health Care")])
display(industry_value_counts[industry_value_counts.index.str.contains("Insurance")])

In [ ]:
highest_day_count = df.groupby("highest_month").count()
highest_day_count["ticker"].plot()
plt.title("Number of stocks that reached new highs")
plt.ylabel("number of stocks")
plt.show()

In [ ]:
highest_day_count["ticker"].plot(marker=".")
plt.grid(axis='y')
plt.title("Number of stocks that reached new highs")
plt.xlim("2021-01-01","2022-04-30")
plt.ylabel("number of stocks")
plt.show()

In [ ]:
tikcer_list = ["GOOG", "AAPL", "FB", "AMZN", "MSFT", "TSLA", "NVDA"]
df[df["ticker"].isin(tikcer_list)]

In [ ]:
industry_value_counts = df[df["highest_day"] <= "2021-12-31"]["GICS Sub-Industry"].value_counts()
fig, ax = plt.subplots(figsize=(20, 8))
ax.bar(industry_value_counts.index, industry_value_counts.values)
ax.set_xticklabels(industry_value_counts.index, rotation=90)
ax.set_xlabel("industry")
ax.set_ylabel("number of stocks")
plt.show()

In [ ]:
df["in_2022"] = df["highest_day"].map(lambda x: False if x.year < 2022 else True)
value_counts_before_2022 = df[df["in_2022"]==False]["GICS Sub-Industry"].value_counts()
value_counts_2022 = df[df["in_2022"]==True]["GICS Sub-Industry"].value_counts()
value_counts_before_2022.name = "~2021"
value_counts_2022.name = "2022"
comparison_df = pd.concat([value_counts_2022, value_counts_before_2022], axis=1)
comparison_df = comparison_df.fillna(0)
comparison_df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 8))
comparison_df.plot(kind='bar', stacked=True, ax=ax)
ax.set_xlabel("industry")
ax.set_ylabel("number of stocks")
plt.show()